# Tag assignement Euristic

This notebooks describes how one can scrip the creation of a `fort-myrmidon` Experiment, associate tracking data and adds Ant and Identification following a custom euristic. Beware that it will have the following shortcoming:

* Created ants will have no head-tail measurement, therfore their orientation will be the default tag orientation. One should either a) manually add the head-tail measurement in `fort-studio`, or b) compute an orientation / center offset with a dedicated euristic and use [fm.Identification.SetUserDefinedAntPose](https://formicidae-tracker.github.io/myrmidon/latest/api/python/ant_identification.html#py_fort_myrmidon.Identification.SetUserDefinedAntPose) to set this offset.
* This notebooks fully destroy the file `dataset/example-automatic.myrmidon`

In most situation, one would prefer to use `fort-studio` to perform these steps.

## 1. Create the experiment file and add the tracking data directory

We start by creating a new experiment file, and associate the tracking data directory to it.

---

**NOTE**

The example tracking dataset  of this directory is available at https://drive.switch.ch/index.php/s/1EuXdsov3mxsRyc (15Go)

---

In [1]:
%load_ext blackcellmagic

In [2]:
import py_fort_myrmidon as fm

exampleTrackingDataDirectory = "dataset/example.0000"
exampleExperimentFilePath = "dataset/example-automatic.myrmidon"

e = fm.Experiment(exampleExperimentFilePath)

s = e.CreateSpace("main")

e.AddTrackingDataDirectory(s.ID, exampleTrackingDataDirectory, fixCorruptedData=True)

e.Save(exampleExperimentFilePath)

## 2. Extract tracking statistics in a DataFrame

The next steps is to extract all tag tracking statistice and put it in a dataframe. At the moment [fm.Query.ComputeTagStatistics]() returns a `dict` of `fm.TagStatistics` which are not so convenient to work with. It may directly return a `pandas.DataFrame` in the future 

In [3]:
import pandas as pd
import numpy as np


# we use here python comprehension which are quite useful with pandas.
# Simply read [f(x) for x in l] as create an array with function 'f' applied to every element of 'l'.
tagStatistics = fm.Query.ComputeTagStatistics(e)
tagStatistics = pd.DataFrame(
    data={
        "tagID[decimal]": tagStatistics.keys(),
        # transforms fm.Time to numpy.datetime64[ns]
        "firstSeen": np.array(
            [str(ts.FirstSeen)[:-1] for ts in tagStatistics.values()],
            dtype="datetime64[ns]",
        ),
        # transforms fm.Time to numpy.datetime64[ns]
        "lastSeen": np.array(
            [str(ts.LastSeen)[:-1] for ts in tagStatistics.values()],
            dtype="datetime64[ns]",
        ),
        "trackingCount": [ts.Counts[0] for ts in tagStatistics.values()],
        "multipleSeen": [ts.Counts[1] for ts in tagStatistics.values()],
    },
    index=["0x%03x" % k for k in tagStatistics.keys()],
)

tagStatistics

,tagID[decimal],firstSeen,lastSeen,trackingCount,multipleSeen
0x001,1,2021-02-15 23:24:48.963376369,2021-02-19 23:08:30.161703312,151357,0
0x005,5,2021-02-15 21:29:44.182873326,2021-02-19 23:29:36.917283461,955547,0
0x006,6,2021-02-15 21:29:41.182818780,2021-02-19 17:44:50.697435769,135206,0
0x007,7,2021-02-15 22:57:59.331374109,2021-02-19 20:57:27.375120698,192812,0
0x00a,10,2021-02-15 21:29:41.182818780,2021-02-19 23:29:39.542389710,2251653,1
...,...,...,...,...,...
0x3d8,984,2021-02-16 09:44:23.754828464,2021-02-19 08:31:00.544960526,52,0
0x3d9,985,2021-02-16 09:14:38.996660753,2021-02-19 22:31:00.776393531,75,0
0x3e4,996,2021-02-15 23:03:27.332740496,2021-02-19 23:28:27.542049504,229,0
0x3e5,997,2021-02-15 21:39:53.435562593,2021-02-19 20:34:59.618847484,169,0


## 3. Consider only tag with a detection ratio of 1% and seen at least for one hour

Now we make a query on the DataFrame to filter out tags that werre not detected for more than one hour and were detected less than 1% of the frames

In [4]:
moreThanOneHour = (tagStatistics['lastSeen'] - tagStatistics['firstSeen']) > np.timedelta64(1,'h')
nbFrames = fm.Query.GetDataInformations(e).Frames
moreThanOnePercent = (tagStatistics['trackingCount'] / nbFrames) > 0.01

filteredTags = tagStatistics[moreThanOneHour * moreThanOnePercent]
filteredTags

,tagID[decimal],firstSeen,lastSeen,trackingCount,multipleSeen
0x001,1,2021-02-15 23:24:48.963376369,2021-02-19 23:08:30.161703312,151357,0
0x005,5,2021-02-15 21:29:44.182873326,2021-02-19 23:29:36.917283461,955547,0
0x006,6,2021-02-15 21:29:41.182818780,2021-02-19 17:44:50.697435769,135206,0
0x007,7,2021-02-15 22:57:59.331374109,2021-02-19 20:57:27.375120698,192812,0
0x00a,10,2021-02-15 21:29:41.182818780,2021-02-19 23:29:39.542389710,2251653,1
...,...,...,...,...,...
0x0e3,227,2021-02-15 21:29:41.182818780,2021-02-19 23:29:39.417294270,1403318,0
0x0e6,230,2021-02-15 21:57:39.065260711,2021-02-19 23:18:56.414705242,474822,0
0x0ea,234,2021-02-15 21:29:41.182818780,2021-02-19 23:29:21.667301635,1124768,0
0x0eb,235,2021-02-15 21:29:41.182818780,2021-02-19 23:29:39.542389710,1480788,0


## 4. Create Ants and Identification from filtered list

We now create a list of ant identified by the selected tags.

---

**WARNING**

You can only run this loop once. Otherwise on the second iteration the Tag IDs will already be consumed by the previously created ants.

---


In [5]:
for tagID in filteredTags['tagID[decimal]']:
    a = e.CreateAnt()
    # in python, start and end already defaults to fm.Time.SinceEver() and fm.Time.Forever()
    e.AddIdentification(tagID=tagID,antID=a.ID)
    
# we save our modified experiment to the disk !!
e.Save(exampleExperimentFilePath)

In [6]:
identificationCorrespondances = pd.DataFrame(
    data={
        "TagID": [
            fm.FormatTagID(tagID)
            for tagID in e.IdentificationsAt(fm.Time.Now()).values()
        ],
        "AntID": [
            fm.FormatAntID(antID) for antID in e.IdentificationsAt(fm.Time.Now()).keys()
        ],
    }
)
identificationCorrespondances

,TagID,AntID
0,0x001,001
1,0x005,002
2,0x006,003
3,0x007,004
4,0x00a,005
...,...,...
109,0x0e3,110
110,0x0e6,111
111,0x0ea,112
112,0x0eb,113
